In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
import requests
import math

### Helper Function

In [2]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees).
    Returns the distance in miles.
    """
    R = 3958.8  
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

### Compute desirable score

In [ ]:
def compute_score(distance, group, is_rural):
    
    points = 0.0
    if group == 1:
        if distance <= 0.5:
            points = 2.5
        elif distance <= 1:
            points = 2.0
        elif (not is_rural) and distance <= 1.5:
            points = 1.5
        elif is_rural and distance <= 2.5:
            points = 2.5
    elif group == 2:
        if distance <= 0.5:
            points = 2.0
        elif distance <= 1:
            points = 1.5
        elif (not is_rural) and distance <= 1.5:
            points = 1.0
        elif is_rural and distance <= 2.5:
            points = 1.0
    return points

In [ ]:
def compute_desirable_score(lihtc_lat, lihtc_lon, is_rural, csv_file):

    df = pd.read_csv(csv_file)

    # Define the mapping of amenity names to their scoring groups.
    amenity_groups = {
        "national_big_box_store": 1,
        "retail_store": 2,
        "grocery_store": 1,
        "restaurant": 2,
        "hospital": 1,
        "medical_clinic": 1,
        "pharmacy": 1,
        "technical_college": 2,
        "school": 1,  
        "town_square": 1,
        "public_park": 1,
        "library": 1,
        "fire_police_station": 2,
        "police station": 2,
        "bank": 2,
        "place_of_worship": 2,
        "post_office": 2
    }

    total_score = 0.0
    scores = {}  

    # Loop through each desirable amenity type
    for amenity, group in amenity_groups.items():
        # Filter to rows matching this amenity.
        df_subset = df[df['amenity_key'].str.lower() == amenity.lower()]
        if df_subset.empty:
            continue

        # Compute the distance for each record in the subset from the LIHTC development site.
        df_subset = df_subset.copy()  # avoid SettingWithCopy warnings
        df_subset['distance'] = df_subset.apply(
            lambda row: haversine(lihtc_lat, lihtc_lon, row['lat'], row['lon']),
            axis=1
        )

        # Find the nearest amenity of this type (minimum distance)
        min_distance = df_subset['distance'].min()

        # Compute the score for this amenity based on the computed distance.
        points = compute_score(min_distance, group, is_rural)
        scores[amenity] = {"distance": min_distance, "points": points}
        total_score += points

    return total_score, scores


In [ ]:
if __name__ == "__main__":

    lihtc_lat = 32.84693   
    lihtc_lon = -83.63868 
    is_rural = False

    csv_file = "../../data/processed/scoring_indicators/desirable_activities_google_places.csv"

    total_score, amenity_details = compute_desirable_score(lihtc_lat, lihtc_lon, is_rural, csv_file)
    print("Total Desirable Activities Score:", total_score)
    print("Individual Amenity Details:")
    for amenity, details in amenity_details.items():
        print(f" - {amenity.title()}: {details['distance']:.2f} miles, {details['points']} points")

Total Desirable Activities Score: 24.0
Individual Amenity Details:
 - National_Big_Box_Store: 1.98 miles, 0.0 points
 - Retail_Store: 0.49 miles, 2.0 points
 - Grocery_Store: 2.23 miles, 0.0 points
 - Restaurant: 0.52 miles, 1.5 points
 - Hospital: 0.56 miles, 2.0 points
 - Medical_Clinic: 2.86 miles, 0.0 points
 - Pharmacy: 0.48 miles, 2.5 points
 - Technical_College: 0.35 miles, 2.0 points
 - School: 0.79 miles, 2.0 points
 - Town_Square: 0.84 miles, 2.0 points
 - Public_Park: 0.32 miles, 2.5 points
 - Library: 0.43 miles, 2.5 points
 - Fire_Police_Station: 0.50 miles, 1.5 points
 - Bank: 0.49 miles, 2.0 points
 - Place_Of_Worship: 2.35 miles, 0.0 points
 - Post_Office: 0.50 miles, 1.5 points


### Compute undesirable score